In [1]:
import pandas as pd
import os
import gc
root_dir = os.path.abspath(os.curdir)
pred_dir_subs = os.path.dirname(root_dir)+"/SubmissionPredictions/"
pred_dir_coms = os.path.dirname(root_dir)+"/CommentPredictions/"
#Create directory to save experiment results
results_dir = os.path.dirname(root_dir)+"/Results/"
os.makedirs(results_dir, exist_ok=True)

In [2]:
df_dtype = {
        "Author": 'category',
        "Probability":'float',
        "Label":'string'}


def getRawCounts(collection,pred_dir):
    print(collection)
    for year in range(2006,2008):
        print(year)
        yearFile = pred_dir+str(year)+'.csv'

        output = results_dir+"rawToxicityCounts-"+collection+"-"+str(year)+".csv"
        with open(output,'a') as out:
            out.write("Username,Class1,Count1,Class2,Count2,Class3,Count3\n")
     
            fields =['Author','Probability','Label']

            df = pd.read_csv(yearFile, usecols=fields,dtype=df_dtype)
            label = df.Label.mode().values

            for user,df1 in df.groupby('Author'):
                HT = 0
                NT = 0
                ST = 0
                out.write(user+',')
                counter = df1.groupby(['Label'])
                if 'non_toxic' in counter.groups.keys():
                    NT = counter.get_group('non_toxic').count()['Probability']
                    NT1 = (NT/float(len(df1)))*100
                else:
                    NT1 = 0
                if 'highly_toxic' in counter.groups.keys():
                    HT = counter.get_group('highly_toxic').count()['Probability']
                    HT1 = (HT/float(len(df1)))*100
                else:
                    HT1 = 0
                if 'slightly_toxic' in counter.groups.keys():
                    ST = counter.get_group('slightly_toxic').count()['Probability']
                    ST1 = (ST/float(len(df1)))*100
                else:
                    ST1 = 0

                out.write("non_toxic"+','+str(NT)+',')

                out.write("highly_toxic"+','+str(HT)+',')

                out.write("slightly_toxic"+','+str(ST)+'\n')
            del df
            gc.collect()
    print("Done!")

In [ ]:
getRawCounts("Submission",pred_dir_subs)
getRawCounts("Comment",pred_dir_coms)

Submission
2006
2007
Done!
Comment
2006
2007
